In [2]:
! pip install arabert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 4.1 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.3 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186456 sha256=20cc4901e9bea279407c34e397173c45d73126269b17667f1d2e77c8fb3f1b42
  Stored in directory: /root/.cache/pip/wheels/94/08/b4/78657b1541bb704b088317b52429ee4016d9888fe47dbb130f
Successfully built emoji
  Attempting uninstall: emoji
    Found existing installation: emoji 2.14.1
    Uninstalling emoji-2.14.1:
      Successfully uninstalled emoji-2.14.1


In [3]:
import os
import numpy as np
import tensorflow as tf
from transformers import AutoTokenizer, AutoConfig, TFAutoModelForSequenceClassification
from arabert.preprocess import ArabertPreprocessor
from sklearn.preprocessing import LabelEncoder

class ArabicSentimentClassifier:
    def __init__(self, model_path, model_name="aubmindlab/bert-base-arabertv2"):
        print("Initializing Arabic Sentiment Classifier...")

        self.arabert_prep = ArabertPreprocessor(model_name=model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        # Load the model
        try:
            config = AutoConfig.from_pretrained(os.path.join(model_path, 'config.json'))
            print("Config loaded successfully.")

            self.model = TFAutoModelForSequenceClassification.from_pretrained(
                model_path, config=config, from_tf=True
            )
            print("Model loaded from directory using config and .h5!")
        except Exception as e:
            print(f"Error loading from directory: {e}")
            print("Attempting fallback loading with model_name and weights...")
            self.model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

            # Manually load weights
            h5_file = next((f for f in os.listdir(model_path) if f.endswith(".h5")), None)
            if not h5_file:
                raise ValueError("No .h5 file found in the provided directory.")
            self.model.load_weights(os.path.join(model_path, h5_file))
            print("Model weights loaded successfully.")

        # Setup label encoder
        self.label_encoder = LabelEncoder()
        self.label_encoder.fit(['0','1', '2'])

    def classify_text(self, text):
        processed_text = self.arabert_prep.preprocess(text)
        encoding = self.tokenizer(
            processed_text,
            truncation=True,
            padding='max_length',
            max_length=512,
            return_tensors='tf'
        )

        output = self.model(encoding)
        probabilities = tf.nn.softmax(output.logits, axis=-1).numpy()[0]
        predicted_idx = np.argmax(probabilities)
        predicted_label = self.label_encoder.classes_[predicted_idx]
        confidence = probabilities[predicted_idx]

        return {
            "original_text": text,
            "processed_text": processed_text,
            "predicted_class": predicted_label,
            "confidence": float(confidence),
            "all_probabilities": {
                label: float(prob) for label, prob in zip(self.label_encoder.classes_, probabilities)
            }
        }

    def classify_batch(self, texts):
        return [self.classify_text(text) for text in texts]



2025-06-12 20:05:26.919856: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749758727.188835      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749758727.267062      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:

MODEL_PATH = "/kaggle/input/fork-of-sentiment-analysis-of-messages/arabic_sentiment_model/"  

classifier = ArabicSentimentClassifier(MODEL_PATH)

examples = [
    "هذه الشركة ممتازة وأنصح بالعمل بها",
    "الإدارة سيئة جداً والراتب قليل",
    "الشركة عادية، لا بأس بها",
    "أحب العمل هنا كثيراً",
    "التجربة مخيبة للآمال"
]

for i, text in enumerate(examples, 1):
    result = classifier.classify_text(text)
    print(f"\nExample {i}: {text}")
    print(f"  Predicted Class: {result['predicted_class']}")
    print(f"  Confidence: {result['confidence']:.2f}")
    for cls, prob in result['all_probabilities'].items():
        print(f"    {cls}: {prob:.4f}")

# Quick function (optional)
def quick_classify(text, model_path):
    return ArabicSentimentClassifier(model_path).classify_text(text)


Initializing Arabic Sentiment Classifier...


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


100%|██████████| 241M/241M [05:59<00:00, 671kiB/s] 


[2025-06-12 20:11:59,111 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


tokenizer_config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Config loaded successfully.
Error loading from directory: ('Keyword argument not understood:', 'from_tf')
Attempting fallback loading with model_name and weights...


model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

2025-06-12 20:12:16.719916: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bi

Model weights loaded successfully.

Example 1: هذه الشركة ممتازة وأنصح بالعمل بها
  Predicted Class: 2
  Confidence: 0.99
    0: 0.0051
    1: 0.0065
    2: 0.9884

Example 2: الإدارة سيئة جداً والراتب قليل
  Predicted Class: 0
  Confidence: 0.99
    0: 0.9866
    1: 0.0035
    2: 0.0100

Example 3: الشركة عادية، لا بأس بها
  Predicted Class: 2
  Confidence: 0.52
    0: 0.0836
    1: 0.3916
    2: 0.5248

Example 4: أحب العمل هنا كثيراً
  Predicted Class: 2
  Confidence: 0.99
    0: 0.0060
    1: 0.0066
    2: 0.9874

Example 5: التجربة مخيبة للآمال
  Predicted Class: 0
  Confidence: 0.95
    0: 0.9494
    1: 0.0122
    2: 0.0384


In [5]:
negative_sentences = [
    "صارلي أسبوع بستنى وما حدا رد عليّ.",
    "أنا دافع كل المستحقات ولسّا الحساب محجوز!",
    "ليش كل مرة بدي أراجع الفرع عشان أحل المشكلة؟",
    "ما في حدا فاهمني أو بيعطيني جواب واضح!",
    "كل شوي بتغيروا الحكي، وأنا تعبت!",
    "هاد مش أول مرة يصير معي هيك، عنجد الوضع صار مزعج!",
    "ليش تأخرتوا بتحويل الراتب؟ هيك بتخربولي التزاماتي!",
    "من الصبح بتصل وما في رد، هاد اسمه بنك؟",
    "كل ما أراجع بقولولي 'راجع بعدين'، وأنا ما عندي وقت!",
    "حسّيت إني بضيع وقتي وولا شي انحل!",
    "قرف يقرفكم"
]
classifier.classify_batch(negative_sentences)

[{'original_text': 'صارلي أسبوع بستنى وما حدا رد عليّ.',
  'processed_text': 'صارلي أسبوع ب+ ستنى و+ ما حد +ا رد علي .',
  'predicted_class': '0',
  'confidence': 0.6818404197692871,
  'all_probabilities': {'0': 0.6818404197692871,
   '1': 0.06753765046596527,
   '2': 0.2506219446659088}},
 {'original_text': 'أنا دافع كل المستحقات ولسّا الحساب محجوز!',
  'processed_text': 'أنا دافع كل ال+ مستحق +ات ولس +ا ال+ حساب محجوز !',
  'predicted_class': '0',
  'confidence': 0.8504119515419006,
  'all_probabilities': {'0': 0.8504119515419006,
   '1': 0.04518069699406624,
   '2': 0.10440736263990402}},
 {'original_text': 'ليش كل مرة بدي أراجع الفرع عشان أحل المشكلة؟',
  'processed_text': 'ل+ يش كل مر +ة بدي أراجع ال+ فرع عشان أحل ال+ مشكل +ة ؟',
  'predicted_class': '0',
  'confidence': 0.7454066872596741,
  'all_probabilities': {'0': 0.7454066872596741,
   '1': 0.11967132240533829,
   '2': 0.13492195308208466}},
 {'original_text': 'ما في حدا فاهمني أو بيعطيني جواب واضح!',
  'processed_text': 'ما

In [6]:
positive_sentences = [
    "عنجد شكراً، خدمتكم ممتازة اليوم.",
    "كل الاحترام لإلك، ساعدتيني كتير.",
    "هي أول مرة بتعامل معكم، والتجربة كانت ممتازة.",
    "بصراحة، كتير مبسوط من سرعة الرد والمتابعة.",
    "بشكر البنك على اهتمامه بحل المشكلة بسرعة.",
    "أنا زبون عندكم من زمان، وما عمري شفت تقصير.",
    "ما قصّرتوا معي، يعطيكم العافية.",
    "واضح إنو في تحسن كبير بخدمتكم، استمروا هيك.",
    "شكراً لإنك وضحتلي الموضوع بكل سهولة.",
    "دايمًا بلاقي حد يساعدني وقت الحاجة، شكراً إلكم."
]
classifier.classify_batch(positive_sentences)

[{'original_text': 'عنجد شكراً، خدمتكم ممتازة اليوم.',
  'processed_text': 'عنجد شكر +ا ، خدم +ت +كم ممتاز +ة ال+ يوم .',
  'predicted_class': '2',
  'confidence': 0.9887514710426331,
  'all_probabilities': {'0': 0.005748237948864698,
   '1': 0.005500297527760267,
   '2': 0.9887514710426331}},
 {'original_text': 'كل الاحترام لإلك، ساعدتيني كتير.',
  'processed_text': 'كل ال+ احترام ل+ إلك ، ساعدتيني كتير .',
  'predicted_class': '2',
  'confidence': 0.9864968657493591,
  'all_probabilities': {'0': 0.006995543371886015,
   '1': 0.006507613696157932,
   '2': 0.9864968657493591}},
 {'original_text': 'هي أول مرة بتعامل معكم، والتجربة كانت ممتازة.',
  'processed_text': 'هي أول مر +ة ب+ تعامل مع +كم ، و+ ال+ تجرب +ة كان +ت ممتاز +ة .',
  'predicted_class': '2',
  'confidence': 0.981604814529419,
  'all_probabilities': {'0': 0.008054552599787712,
   '1': 0.010340603068470955,
   '2': 0.981604814529419}},
 {'original_text': 'بصراحة، كتير مبسوط من سرعة الرد والمتابعة.',
  'processed_text': 'ب+ 

In [8]:
import time 


start_time = time.time()
neutral_sentences = [
    "بدي أستفسر عن الرصيد بحسابي."
 
]

classifier.classify_batch(neutral_sentences)

end_time = time.time()

print(end_time-start_time)

1.3104887008666992
